# ![NRGPy](https://www.gravatar.com/avatar/6282094b092c756acc9f7552b164edfe?s=24) nrgpy Example Notebook

In [ ]:
import nrgpy
import pandas as pd

## API Credentials

Retrieve your NRG Cloud API Credentials and paste them in the cell below.

https://cloud.nrgsystems.com/data-manager/api-setup

Alternately, put the credentials in a file named `creds.py` and replace the cell below with:

```python notebook
from creds import client_id, client_secret
```

In [1]:
client_id = "your-client-id-here"
client_secret = "your-client-secret-here"

In [ ]:
sites = nrgpy.cloud_sites(
    client_id=client_id,
    client_secret=client_secret,
)

In [ ]:
sites.sites_df

## Exporting data

Using information from the site table, we can create a text export and save it locally.

In [ ]:
exporter = nrgpy.cloud_export(
    client_id=client_id,
    client_secret=client_secret,
    site_number=110,
    start_date="2022-04-01",
    end_date="2022-05-15",
)

In [ ]:
exporter.export()

## Reading data

nrgpy includes classes for reading in text data. The "reader" object includes organized metadata and time series data. The time series data is a Pandas dataframe object.

In [ ]:
# reader = nrgpy.sympro_txt_read(filename=exporter.export_filepath)
reader = nrgpy.sympro_txt_read(filename="000110_SymPRO_Crow's_Nest_meas_2022.04.01-2022.05.15.txt")

In [ ]:
print(f"This reader is for site {reader.site_number}")
print(f"It is located at {reader.latitude}, {reader.longitude}")

## Quality

There are a few quality checks built in to nrgpy. One useful function is `check_intervals`.

In [ ]:
nrgpy.check_intervals(reader.data, return_info=True, verbose=False)

In [ ]:
reader.ch_info #.head()

## Plots

Below are some examples of plots using the [Matplotlib](https://matplotlib.org) and [Windrose](https://github.com/python-windrose/windrose/) libraries.

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib as mpl
from windrose import plot_windrose

plt.rcParams["image.cmap"] = "plasma"
# plt.rcParams["axes.prop_cycle"]

In [ ]:
# get the wind speed channels from the ch_info dataframe

ws_channels = reader.ch_info['Channel:'].loc[reader.ch_info['Type:'] == 'Anemometer']

ws_channels

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

for ch in ws_channels:
    plot_col = [col for col in reader.data.columns if f"Ch{ch}_" in col and "avg" in col.lower()]
    ax.plot(
        reader.data['Timestamp'],
        reader.data[plot_col],
        label=plot_col,
    )
    
plt.title(f"Wind speed plot for site {reader.site_description}")
plt.ylabel("m/s")
plt.legend()

## Strip plot of met sensors

In [ ]:
met_units = ["C", "%RH", "mb"]
ax_list = []

for i, unit in enumerate(met_units):
    plot_channels = reader.ch_info['Channel:'].loc[reader.ch_info['Units:'] == unit]
    plot_col = [col for col in reader.data.columns if f"Ch{ch}_" in col and "avg" in col.lower()]
    ax_list.append(plt.subplot(int(f"{len(met_units)}1{1+i}")))
    
    plt.plot(
        reader.data['Timestamp'],
        reader.data[plot_col].values,
        label=plot_col
    )
    plt.ylabel(unit)
    if i == 0:
        plt.title(f"Strip plot for {reader.site_description}", loc="center")


In [ ]:
reader.ch_info.loc[(reader.ch_info['Type:'] == "Anemometer") | (reader.ch_info['Type:'] == "Vane")]

In [ ]:
anem_col = [col for col in reader.data.columns if "Ch1_" in col and "avg" in col.lower()]
vane_col = [col for col in reader.data.columns if "Ch13_" in col and "avg" in col.lower()]

windrose_df = pd.DataFrame()
windrose_df['speed'] = reader.data[anem_col]
windrose_df['direction'] = reader.data[vane_col]

plot_windrose(windrose_df, kind='bar', bins=np.arange(0.01,8,1), cmap=cm.plasma, lw=3)

plt.title(f"Wind rose diagram for {reader.site_description}")
plt.show()